In [1]:
@file:DependsOn("ro.jf.funds:funds-notebook-client:1.0.0")
%use dataframe
%use kandy

import ro.jf.funds.client.notebook.*
import ro.jf.funds.reporting.api.model.*

val username = "Johann-38.9"

val monthlyReportFrom = YearMonthTO(2019, 1)
val monthlyReportTo = YearMonthTO(2022, 2)
val monthlyReportForecastUntil = YearMonthTO(2022, 8)

val yearlyReportFrom = 2019
val yearlyReportTo = 2021
val yearlyReportForecastUntil = 2023

val fundName = "Expenses"
val reportViewName = "Expenses report"

val REPORT_DATA_CONFIGURATION_YAML_FILE = "../../data/provision/expenses-report-data-configuration.yaml"

val client = FundsClient()

In [2]:
val user = client.ensureUserExists(username)
user

UserTO(id=97726e7d-975a-4727-9274-5afa332f8221, username=Johann-38.9)

In [3]:
val reportView = client.createReportView(user, reportViewName, fundName, client.fromYaml(File(REPORT_DATA_CONFIGURATION_YAML_FILE)))
reportView

ReportViewTO(id=b3302e57-7ab5-4ef7-b725-a3f929417b3d, name=Expenses report, fundId=12d9ce2a-dd68-414a-a067-2f3f117bf003, dataConfiguration=ReportDataConfigurationTO(currency=Currency(value=RON), groups=[ReportGroupTO(name=basic, filter=RecordFilterTO(labels=[basic])), ReportGroupTO(name=home, filter=RecordFilterTO(labels=[home])), ReportGroupTO(name=shopping_services, filter=RecordFilterTO(labels=[shopping_services])), ReportGroupTO(name=transport, filter=RecordFilterTO(labels=[transport])), ReportGroupTO(name=fun, filter=RecordFilterTO(labels=[fun])), ReportGroupTO(name=gifts, filter=RecordFilterTO(labels=[gifts])), ReportGroupTO(name=development, filter=RecordFilterTO(labels=[development])), ReportGroupTO(name=investment, filter=RecordFilterTO(labels=[investment]))], reports=ReportsConfigurationTO(net=NetReportConfigurationTO(enabled=false, filter=null), valueReport=ValueReportConfigurationTO(enabled=false, filter=null), groupedNet=GenericReportConfigurationTO(enabled=false), grouped

In [4]:
val monthlyReportData = client.getMonthlyReportViewData(user, reportViewName, monthlyReportFrom, monthlyReportTo, monthlyReportForecastUntil)


In [5]:
val yearlyData = client.getYearlyReportViewData(user, reportViewName, yearlyReportFrom, yearlyReportTo, yearlyReportForecastUntil)


In [6]:
import kotlinx.datetime.DateTimeUnit
import kotlinx.datetime.TimeZone
import kotlinx.datetime.atStartOfDayIn
import kotlinx.datetime.minus
import org.jetbrains.kotlinx.kandy.ir.Plot
import java.math.BigDecimal

fun plotGroupData(
    reportData: ReportDataTO,
    plotTitle: String,
    groupItemFilter: (ReportDataGroupedBudgetItemTO) -> Boolean,
): Plot {
    val dataFrame = dataFrameOf(
        "bucket" to reportData.data.map<ReportDataItemTO, LocalDate> { it.timeBucket.from },
        "spent" to reportData.data.map { bucket ->
            bucket.groupedBudget!!.filter(groupItemFilter).map { it.spent!! }.reduce { acc, value -> acc + value }
        },
        "allocated" to reportData.data.map { bucket ->
            bucket.groupedBudget!!.filter(groupItemFilter).map { it.allocated!! }.reduce { acc, value -> acc + value }
        },
        "left" to reportData.data.map { bucket ->
            bucket.groupedBudget!!.filter(groupItemFilter).map { it.left!! }.reduce { acc, value -> acc + value }
        },
        "net" to reportData.data.map { bucket ->
            bucket.groupedBudget!!.filter(groupItemFilter).map { it.allocated!! + it.spent!! }
                .reduce { acc, value -> acc + value }
        },
    )
    return dataFrame
        .plot {
            line {
                val values =
                    listOf("spent", "allocated", "left").flatMap { dataFrame.get(it).values }.map { it as BigDecimal }
                val forecastBorderMin = minOf(BigDecimal.ZERO, values.minOrNull() ?: BigDecimal.ZERO)
                val forecastBorderMax = values.maxOrNull() ?: BigDecimal.ZERO
                val forecastBorderX = when (reportData.interval.granularity) {
                    TimeGranularityTO.YEARLY -> reportData.interval.toDate.minus(183, DateTimeUnit.DAY)
                    TimeGranularityTO.MONTHLY -> reportData.interval.toDate.minus(15, DateTimeUnit.DAY)
                    TimeGranularityTO.DAILY -> reportData.interval.toDate.minus(1, DateTimeUnit.DAY)
                }

                y(listOf(forecastBorderMin, forecastBorderMax))
                x.constant(forecastBorderX.atStartOfDayIn(TimeZone.UTC).toEpochMilliseconds())
            }
            x("bucket") {
                val format = when (reportData.interval.granularity) {
                    TimeGranularityTO.YEARLY -> "%Y"
                    TimeGranularityTO.MONTHLY -> "%b %Y"
                    TimeGranularityTO.DAILY -> "%d %b %Y"
                }
                axis.breaks(reportData.data.map {
                    it.timeBucket.from.atStartOfDayIn(TimeZone.UTC).toEpochMilliseconds()
                }, format)
            }
            line {
                y.constant(0)
            }
            line {
                y("spent")
                color = Color.RED
            }
            line {
                y("allocated")
                color = Color.GREEN
            }
            area {
                y("left")
                borderLine {
                    color = Color.ORANGE
                }
            }
            layout {
                title = plotTitle
                size = 2400 to 1200
            }
        }
}

fun plotGroup(reportData: ReportDataTO, title: String, group: String): Plot =
    plotGroupData(reportData, title) { it.group == group }

fun plotGroupsTotal(reportData: ReportDataTO, title: String): Plot =
    plotGroupData(reportData, title) { true }



In [7]:
plotGroupsTotal(yearlyData, "Yearly total expenses")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="2HtZGm"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Yearly total expenses"
},
"mapping":{
},
"data":{
"bucket":[1.5463008E12,1.5778368E12,1.6094592E12,1.6409952E12,1.6725312E12],
"left":[18161.1761693326,3543.9099255863007,93867.2333536247,123253.14207673556,152639.05079984642],
"spent":[-103911.88029852,-186186.29,-125298.11,-138465.42676617333,-138465.42676617333],
"allocated":[116306.4464678526,171597.41,215650.15,167851.3354892842,167851.3354892842]
},
"ggsize":{
"width":2400.0,
"height":1200.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5778368E12,1.6094592E12,1.6409952E12,1.6725312E12],
"format":"%Y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5778368E12,1.6094592E12,1.6409952E12,1.6725312E12],
"format":"%Y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5778368E12,1.6094592E12,1.6409952E12,1.6725312E12],
"format":"%Y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5778368E12,1.6094592E12,1.6409952E12,1.6725312E12],
"format":"%Y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"y":"y"
},
"stat":"identity",
"data":{
"y":[-186186.29,215650.15]
},
"sampling":"none",
"x":1.6250976E12,
"inherit_aes":false,
"position":"identity",
"geom":"line"
},{
"mapping":{
"x":"bucket"
},
"stat":"identity",
"sampling":"none",
"y":0.0,
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"bucket",
"y":"spent"
},
"stat":"identity",
"color":"#ee6666",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"bucket",
"y":"allocated"
},
"stat":"identity",
"color":"#3ba272",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"bucket",
"y":"left"
},
"stat":"identity",
"color":"#fc8452",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"area",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"datetime",
"column":"bucket"
}]
},
"spec_id":"2"
};
 var containerDiv = document.getElementById("2HtZGm");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 2400.0,
 height: 1200.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2019 
 
 
 
 
 
 
 
 
 2020 
 
 
 
 
 
 
 
 
 2021 
 
 
 
 
 
 
 
 
 2022 
 
 
 
 
 
 
 
 
 2023 
 
 
 
 
 
 
 
 
 
 
 -200,000 
 
 
 
 
 
 
 -180,000 
 
 
 
 
 
 
 -160,000 
 
 
 
 
 
 
 -140,000 
 
 
 
 
 
 
 -120,000 
 
 
 
 
 
 
 -100,000 
 
 
 
 
 
 
 -80,000 
 
 
 
 
 
 
 -60,000 
 
 
 
 
 
 
 -40,000 
 
 
 
 
 
 
 -20,000 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 20,000 
 
 
 
 
 
 
 40,000 
 
 
 
 
 
 
 60,000 
 
 
 
 
 
 
 80,000 
 
 
 
 
 
 
 100,000 
 
 
 
 
 
 
 120,000 
 
 
 
 
 
 
 140,000 
 
 
 
 
 
 
 160,000 
 
 
 
 
 
 
 180,000 
 
 
 
 
 
 
 200,0

In [8]:
plotGroupsTotal(monthlyReportData, "Monthly total expenses")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="iD0EVm"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Monthly total expenses"
},
"mapping":{
},
"data":{
"bucket":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12,1.6460928E12,1.6487712E12,1.6513632E12,1.6540416E12,1.6566336E12,1.659312E12],
"left":[8248.12,10951.86,17068.04,17292.8960226764,16410.270981426,17635.2974648376,14404.6527559468,-783.959311916801,4894.493059188699,9818.4534872447,11803.909861618202,18161.1761693326,24395.4607953271,29941.4728379681,27812.2885328405,29842.138180909904,23411.7245814526,12509.992806596007,-4729.583404812095,-2051.627116044803,-55.8684978775985,-320.5856971856079,2719.2896873382756,3543.9099255862616,1941.6889890445634,9125.554940831056,15143.57418195275,21279.293573135456,19211.898112079863,364.05262360674567,23467.587072612027,76474.37636629674,66351.38110734883,110995.67889443733,100390.43768464203,93867.23335362463,80498.38271640743,130568.68703741144,140691.1237040781,151154.235092967,161977.7732642633,173875.7929498343,188335.79677586953,202075.7375874077],
"spent":[-4468.51,-4301.32,-9171.45,-6968.18282172,-9043.57,-7024.09,-11232.58,-25034.7199255,-4539.6390361,-6776.5,-9787.68,-5529.22,-4702.18,-5442.03,-35747.94,-17363.44,-18069.43,-20971.82,-27334.72,-15561.74,-8102.56,-10342.12,-8651.84,-13896.47,-15195.09,-6317.14,-7060.76,-7597.91,-16297.24,-18845.79,-16242.68,-3827.13,-10017.29,-6768.53,-10605.61,-6522.94,-13377.18,-14533.9,-10974.746666666668,-11300.912222222223,-11609.49574074074,-11218.850385802469,-10583.27208461934,-10111.65475833762],
"allocated":[6950.02,7005.06,15287.63,7193.0388443964,8165.27,8255.99,7999.52,9846.94,10218.9435174615,11703.85,11782.42,11887.54,10934.13,10999.45,33626.2,19395.19,11641.91,10066.59,10094.99,18240.71,10110.61,10074.3,11694.38,14718.95,13596.0,13500.0,13095.1,13735.9,14227.0,0.0,39343.44,56840.05,-100.34,51413.0,0.0,0.0,6.97,64605.08,21097.183333333334,21764.02361111111,22433.033912037034,23116.870071373458,25043.275910654582,23851.595569875793]
},
"ggsize":{
"width":2400.0,
"height":1200.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12,1.6460928E12,1.6487712E12,1.6513632E12,1.6540416E12,1.6566336E12,1.659312E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808

In [9]:
plotGroup(monthlyReportData, "Basic Expenses", "basic")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="6vdB6i"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Basic Expenses"
},
"mapping":{
},
"data":{
"bucket":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12,1.6460928E12,1.6487712E12,1.6513632E12,1.6540416E12,1.6566336E12,1.659312E12],
"left":[1510.0,1102.1899999999998,2340.5599999999995,2060.6230060459993,2238.1176337514466,2166.050188007877,616.6168454230186,781.491224085394,1953.6330206118023,2624.990075660107,2575.17807325847,3029.168196437852,3276.9475971433567,3529.7652104928425,8601.767610068911,11725.909870384527,12217.70288461522,11179.388477397883,10157.185908525049,9817.782940319776,9505.461678835802,7356.201421798139,7872.534037722407,7638.060177300573,7837.3221836385,8705.792549792137,9230.512522328501,9888.96851067646,10766.07043219382,8013.708566118699,12851.141126288048,21030.373875633435,18146.148435009065,26749.221337254618,23251.5902132443,20104.0789911136,18270.740133568608,29022.0813804653,30711.8113804653,32497.328047131967,34376.65110268752,36339.602746206045,38695.40119335111,40849.486177758256],
"spent":[-2305.04,-2509.31,-3347.92,-2293.91,-2108.18,-2382.69,-3709.61,-2493.78,-1587.87,-2253.23,-2992.86,-2517.62,-2158.1,-2165.51,-2324.86,-948.25,-1951.84,-3154.12,-3041.34,-3989.72,-2393.26,-3636.25,-1883.73,-3184.42,-2513.19,-1835.59,-2078.71,-2087.36,-1969.62,-2751.21,-3092.31,-3185.44,-2862.68,-1679.48,-3497.72,-3147.45,-1835.02,-2169.46,-2529.705,-2567.287916666667,-2607.281909722222,-2660.420402199074,-2652.85460238233,-2616.233319247524],
"allocated":[2085.01,2101.5,4586.29,2013.973006046,2286.19,2311.56,2159.88,2658.69,2759.1623689134,2925.94,2945.53,2971.84,2405.49,2419.86,7397.74,4072.98,2444.77,2113.98,2019.0,3648.14,2022.12,2014.9,2338.92,2943.75,2719.2,2700.0,2619.0,2747.2,2845.4,0.0,7868.67,11368.03,-20.07,10282.6,0.0,0.0,1.39,12921.0,4219.435,4352.804583333333,4486.604965277778,4623.372045717593,5008.653049527392,4770.318303654675]
},
"ggsize":{
"width":2400.0,
"height":1200.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12,1.6460928E12,1.6487712E12,1.6513632E12,1.6540416E12,1.6566336E12,1.659312E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.622

In [10]:
plotGroup(monthlyReportData, "Home Expenses", "home")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="JCdRw2"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Home Expenses"
},
"mapping":{
},
"data":{
"bucket":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12,1.6460928E12,1.6487712E12,1.6513632E12,1.6540416E12,1.6566336E12,1.659312E12],
"left":[3315.04,4807.38,7629.770000000001,9531.584864054,11742.120966199953,13911.882725463067,16293.61806597269,858.0967999749093,4012.400358399721,8000.301660071118,12070.057148449443,16186.259698159398,20161.960447902293,21815.732186350146,5562.498172379423,-2622.051900771636,-10715.347657328653,-23660.538765289097,-42909.83963043959,-45969.94667438695,-45562.76450214935,-41251.56921370735,-40013.070310906136,-37261.88291130767,-41801.4031720171,-38364.209676450286,-34245.316532400895,-30763.551831917674,-31580.95419217781,-43596.34533356985,-28141.26355567332,-9299.891891368527,-11191.899433243918,9647.90961497715,6048.671670770648,5085.825744437688,3574.099396147552,31532.140516313386,37377.76551631339,43373.00259964672,49578.12360659116,56368.11219744764,64725.49733754217,72463.7204059779],
"spent":[-500.0,-609.16,-1763.9,-400.0,-400.0,-467.23,-420.0,-18881.0,-423.24,-456.78,-400.0,-400.0,-400.0,-2736.62,-29698.21,-16136.26,-12866.69,-17070.89,-23488.88,-10712.07,-3563.87,-2752.85,-3681.19,-3840.81,-10690.64,-2616.15,-1842.52,-2704.57,-7215.39,-12018.77,-1917.23,-6744.06,-1847.5,-2296.12,-3599.27,-962.83,-1514.94,-1114.29,-3648.124166666667,-3798.591180555556,-3889.759612268519,-3612.623746624228,-2912.111558842914,-2995.018355413157],
"allocated":[2085.01,2101.5,4586.29,2301.814864054,2612.92,2641.91,2799.83,3446.42,3576.6108657802,4447.46,4477.32,4517.28,4373.62,4399.82,13450.4,7952.09,4773.14,4127.3,4239.87,7661.16,4246.46,4533.6,5262.56,6623.49,6118.2,6075.0,5892.8,6181.2,6402.1,0.0,17704.56,25578.08,-45.16,23135.9,0.0,0.0,3.14,29072.37,9493.749166666666,9793.828263888889,10094.88061921296,10402.61233748071,11269.49669893744,10733.24142384889]
},
"ggsize":{
"width":2400.0,
"height":1200.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12,1.6460928E12,1.6487712E12,1.6513632E12,1.6540416E12,1.6566336E12,1.659312E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1

In [11]:
plotGroup(monthlyReportData, "Transport Expenses", "transport")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="1IWdE3"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Transport Expenses"
},
"mapping":{
},
"data":{
"bucket":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12,1.6460928E12,1.6487712E12,1.6513632E12,1.6540416E12,1.6566336E12,1.659312E12],
"left":[431.57000000000005,574.0,124.26000000000002,346.0401529524,362.8836065501172,592.5216111021381,944.6327615438677,257.9981910552733,744.2571143438095,1675.7116956879047,350.02726285834314,-481.6539828077918,112.56410039242976,1161.2914624665382,4629.382732248241,6956.486416803637,7133.911891231831,8140.157854136952,8864.127923212418,11237.257808524664,12174.530678942006,12503.766811638516,11338.954500391736,9463.008913346732,11189.158633028283,13038.915521294031,14566.625844010034,17036.94174211425,19360.019178821087,19124.137842817276,25559.123105382776,35625.09600317577,33819.756631828364,42712.97885371947,42160.500770309125,41870.22976045281,41431.236115088075,52658.39541704893,55950.73791704894,59388.189792048935,62906.05432329894,66523.6742321531,70462.24663341178,74204.93006810869],
"spent":[-204.24,-207.85,-1214.12,-281.75,-554.64,-348.18,-288.04,-1474.32,-331.47,-238.42,-2502.33,-2020.41,-608.51,-161.2,-230.49,0.0,-1218.9,-202.81,-588.5,0.0,-444.43,-405.02,-2905.92,-3945.53,-304.89,-181.04,-413.17,0.0,-240.1,-233.81,-792.5,-158.6,-1784.78,-360.99,-552.62,-290.16,-440.85,-401.27,-472.4041666666667,-477.3403472222222,-517.1187094907407,-540.2036019483025,-565.736402110661,-546.8394356198828],
"allocated":[347.5,350.28,764.38,503.5301529524,571.57,577.97,640.07,787.74,817.4484968668,1170.39,1178.23,1188.76,1202.79,1209.98,3698.87,2327.42,1397.0,1207.99,1312.37,2371.23,1314.38,1410.4,1637.18,2060.66,2039.4,2025.0,1964.2,2472.5,2560.9,0.0,7081.74,10231.25,-18.06,9254.3,0.0,0.0,1.25,11628.88,3764.746666666667,3914.792222222222,4034.983240740741,4157.823510802469,4504.308803369342,4289.522870316787]
},
"ggsize":{
"width":2400.0,
"height":1200.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12,1.6460928E12,1.6487712E12,1.6513632E12,1.6540416E12,1.6566336E12,1.659312E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.62

In [12]:
plotGroup(monthlyReportData, "Shopping & Services Expenses", "shopping_services")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="HSrKLB"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Shopping & Services Expenses"
},
"mapping":{
},
"data":{
"bucket":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12,1.6460928E12,1.6487712E12,1.6513632E12,1.6540416E12,1.6566336E12,1.659312E12],
"left":[250.23999999999998,600.52,755.1299999999999,-294.85383759480004,-741.880093150332,-2741.229353301329,-2644.884785975403,-1955.421993630142,-1325.1837384315527,-847.4660120059323,-21.894708908976455,810.2672455523964,1316.4214057101044,1975.7558125311598,2733.10458321837,3896.6278362498447,4594.790005407969,5130.476567812616,5635.289638094055,6118.587473101223,6660.774180512197,6693.7509586309125,7140.107478145387,6841.397155662908,7306.152989529819,7039.948167121971,6256.937448278425,5186.819290801951,5216.682646031039,4277.354511119843,3676.9522763855357,5079.3317040662705,2893.875350328333,5368.267839366371,5058.065675777799,4805.3923577399755,4015.031948280726,2512.6182718859122,2133.8482718859123,1787.7157718859123,1501.8372302192454,1189.7046434136903,929.7918410410053,700.9696384705964],
"spent":[-385.57,0.0,-609.77,-1481.52282172,-936.99,-2494.99,-463.18,0.0,-82.99,-342.57,0.0,0.0,-149.99,0.0,-1259.78,0.0,0.0,-69.0,0.0,-429.95,0.0,0.0,-77.0,-893.12,-209.0,-944.99,-1425.22,-1755.98,-682.13,-938.77,-2600.22,-1438.67,-2180.08,-96.3,-310.22,-252.66,-790.78,-4732.84,-1433.655833333333,-1434.358819444444,-1407.557054398148,-1468.009308931327,-1512.112584675604,-1421.436966731905],
"allocated":[347.5,350.28,764.38,431.5389841252,489.89,495.33,559.9,689.31,715.3536654211,819.37,824.77,832.16,656.04,659.95,2017.62,1163.71,698.54,604.0,504.75,912.04,505.53,402.9,467.72,588.79,679.8,675.0,654.8,686.8,711.3,0.0,1967.22,2842.01,-5.02,2570.7,0.0,0.0,0.35,3230.47,1054.885833333333,1088.226319444444,1121.678512731481,1155.876722125772,1252.199782302919,1192.614764161496]
},
"ggsize":{
"width":2400.0,
"height":1200.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12,1.6460928E12,1.6487712E12,1.6513632E12,1.6540416E12,1.6566336E12,1.659312E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.63304

In [13]:
plotGroup(monthlyReportData, "Fun Expenses", "fun")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="vIvfIS"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Fun Expenses"
},
"mapping":{
},
"data":{
"bucket":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12,1.6460928E12,1.6487712E12,1.6513632E12,1.6540416E12,1.6566336E12,1.659312E12],
"left":[876.3500000000004,1357.1000000000008,2412.3500000000004,1238.4191370776007,63.80940215257002,1137.0726752944524,1147.1184062385223,150.93221285702106,-583.0596878220119,-51.875902846608064,890.8031472875796,1741.7436285166564,1818.9975303120564,2478.096919354714,3207.720793698394,4177.221617257883,4758.956579048776,5262.9976715402,5666.822370938003,6047.68696448662,5869.3389040551765,6047.070075378523,6682.061570664717,7423.310739562984,8096.552470492123,8775.747138880386,8711.85199990426,8899.945755570636,9470.235468105866,9469.222243919025,8772.26728966248,9558.095609895585,7103.41503681696,7336.2566001278565,5496.280975307144,5001.266503435365,-2129.4710694214377,-5396.467904569858,-6581.857904569858,-7862.013737903191,-9264.632557347635,-10831.56127841245,-12529.067392899333,-14303.939016926788],
"spent":[-1031.16,-570.0,-1237.89,-2109.0,-2235.76,0.0,-790.09,-1980.8199255,-1756.0690361,-405.5,0.0,-100.0,-579.01,0.0,-1287.38,0.0,0.0,0.0,0.0,-350.0,-619.0,0.0,0.0,0.0,0.0,0.0,-703.0,-360.0,0.0,0.0,-2342.87,-1485.44,-2450.0,-1823.64,-1840.0,-495.0,-7131.09,-5851.14,-2040.181666666667,-2151.613472222222,-2300.914594907407,-2492.657477816358,-2700.378934301054,-2730.171345492809],
"allocated":[1042.5,1050.75,2293.14,935.0691370776,1061.46,1073.29,799.98,984.7,1021.9130824088,936.28,942.63,951.02,656.04,659.95,2017.62,969.72,582.14,503.33,403.76,729.65,404.42,503.7,584.73,735.99,679.8,675.0,654.8,549.4,569.1,0.0,1573.75,2273.56,-4.01,2056.5,0.0,0.0,0.28,2584.12,854.7916666666667,871.4576388888889,898.295775462963,925.7287567515432,1002.872819814172,955.2997214653528]
},
"ggsize":{
"width":2400.0,
"height":1200.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12,1.6460928E12,1.6487712E12,1.6513632E12,1.6540416E12,1.6566336E12,1.659312E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.635724

In [14]:
plotGroup(monthlyReportData, "Gifts Expenses", "gifts")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="mzWPXL"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Gifts Expenses"
},
"mapping":{
},
"data":{
"bucket":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12,1.6460928E12,1.6487712E12,1.6513632E12,1.6540416E12,1.6566336E12,1.659312E12],
"left":[1525.9200000000003,1974.5200000000007,3661.2099999999996,4122.4255740240005,3171.274534420456,2830.696231667663,-1850.6360824736305,-972.2091730805391,-200.24798423047037,833.1706930423303,-1999.727093426,-1420.9585850303624,-1094.311249788349,-324.03952114424595,2896.721044333674,4637.793117815598,3796.52342096186,4328.750704830356,5122.303919287461,6867.471803937303,6837.675107955877,6775.827238179276,7663.695951960759,8003.617723887745,7709.976771260932,8012.1011622013375,8485.331779273429,8521.459499102035,2901.6986169644315,-1.8418369024788073,-3580.105873561552,7879.469398463671,8982.656593776617,10527.133279575479,9845.668256645538,8470.80233270729,6806.705004837065,9125.850960012764,9219.860960012764,9281.07512667943,9344.273807234986,9881.147377836838,10704.696245988845,11895.061686486852],
"spent":[0.0,-392.0,-147.85,-402.0,-1930.0,-1330.0,-5561.66,-204.8,-351.0,-20.0,-3892.49,-491.19,-766.57,-330.2,-141.92,-198.28,-2005.0,-475.0,-216.0,-80.0,-1082.0,-488.0,-104.0,-843.59,-1102.37,-511.87,-298.14,-512.0,-6190.0,-2903.23,-5152.0,9185.08,1107.75,-512.0,-681.5,-1374.84,-1664.5,-264.9,-771.69,-811.1525,-836.081875,-389.92203125,-180.4797005208333,233.8136577690972],
"allocated":[834.0,840.6,1834.54,863.215574024,979.88,990.75,879.94,1083.12,1124.0179138545,1053.28,1060.38,1069.78,1093.4,1099.96,3362.6,1939.55,1164.18,1006.66,1009.5,1824.07,1011.06,805.9,935.55,1177.48,815.8,810.0,785.7,549.4,569.1,0.0,1573.75,2273.56,-4.01,2056.5,0.0,0.0,0.28,2584.12,865.7,872.3666666666667,899.2805555555556,926.7956018518519,1004.02856867284,956.5517827289095]
},
"ggsize":{
"width":2400.0,
"height":1200.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12,1.6460928E12,1.6487712E12,1.6513632E12,1.6540416E12,1.6566336E12,1.659312E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.63831

In [15]:
plotGroup(monthlyReportData, "Development Expenses", "development")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="FZTKWg"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Development Expenses"
},
"mapping":{
},
"data":{
"bucket":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12,1.6460928E12,1.6487712E12,1.6513632E12,1.6540416E12,1.6566336E12,1.659312E12],
"left":[339.00000000000006,536.15,144.76000000000002,288.65712611720005,-426.0550684982098,-261.6966133962672,-101.81245478226619,95.15342682128247,292.6939763174012,-2416.3787223642203,-2060.5339679006574,-1703.650031495547,-1197.119036344791,-695.1292320830539,181.09359689348784,1070.1512231700501,1625.1874575155978,2128.7602961670964,2734.5264655705087,3829.5325679725606,4459.115453970693,1554.3670108963756,2035.0064593594057,1436.3981271329897,1603.9291131120065,1917.2600779914794,2137.6311205589964,2508.710606787798,3078.14596214143,3077.8166301042315,4329.47270412806,6601.901666430533,6597.428492833415,8653.911369416384,8529.660122587475,8529.637663737903,8530.041187906842,11114.068396254996,11878.957562921662,12688.937493477217,13535.46575157907,14405.113031189409,15347.230917433943,16265.508627532188],
"spent":[-42.5,-13.0,-850.0,0.0,-878.0,-1.0,0.0,0.0,-7.0,-3060.0,0.0,0.0,-40.0,-48.5,-805.3,-80.65,-27.0,0.0,0.0,0.0,0.0,-3060.0,0.0,-1189.0,-375.0,-227.5,-300.0,-178.0,0.0,0.0,-345.55,0.0,0.0,0.0,-124.28,0.0,0.0,0.0,-78.98583333333333,-60.56798611111111,-50.7819849537037,-55.01381703317901,-59.59830178594393,-35.76899360143926],
"allocated":[208.5,210.15,458.61,143.8971261172,163.36,165.18,159.92,196.96,204.4371242167,351.13,353.56,356.7,546.75,549.93,1681.35,969.72,582.14,503.33,605.74,1094.42,606.64,402.9,467.72,588.79,543.8,540.0,523.8,549.4,569.1,0.0,1573.75,2273.56,-4.01,2056.5,0.0,0.0,0.28,2584.12,843.875,870.5479166666667,897.3102430555556,924.6610966435185,1001.716188030478,954.0467036996849]
},
"ggsize":{
"width":2400.0,
"height":1200.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12,1.6460928E12,1.6487712E12,1.6513632E12,1.6540416E12,1.6566336E12,1.659312E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12,1.6460928E12,1

In [16]:
plotGroup(monthlyReportData, "Investment Expenses", "investment")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="B3dG2d"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Investment Expenses"
},
"mapping":{
},
"data":{
"bucket":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12,1.6460928E12,1.6487712E12,1.6513632E12,1.6540416E12,1.6566336E12,1.659312E12],
"left":[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],
"spent":[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],
"allocated":[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
},
"ggsize":{
"width":2400.0,
"height":1200.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12,1.6460928E12,1.6487712E12,1.6513632E12,1.6540416E12,1.6566336E12,1.659312E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12,1.6460928E12,1.6487712E12,1.6513632E12,1.6540416E12,1.6566336E12,1.659312E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12,1.6460928E12,1.6487712E12,1.6513632E12,1.6540416E12,1.6566336E12,1.659312E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.